# Определение возраста покупателей

***Описание проекта***   
Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:  

Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
Контролировать добросовестность кассиров при продаже алкоголя.
Постройте модель, которая по фотографии определит приблизительный возраст человека. В вашем распоряжении набор фотографий людей с указанием возраста.  

***План работы над проектом***    
Провести исследовательский анализ набора фотографий.  
Подготовить данные к обучению.  
Обучить нейронную сеть и рассчитайте её качество.  

## Исследовательский анализ данных

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345)

In [ ]:
labels.shape

In [ ]:
labels.hist(bins=100)
plt.show()

In [ ]:
print(labels.describe())

In [ ]:
features, target = next(train_gen_flow)

fig = plt.figure(figsize=(10,10))
for i in range(12):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

***Вывод***    
Набор данных включает 7591 фотографию, что представляет собой небольшую выборку для оценки возраста.

Большинство фотографий попадают в возрастной диапазон от 20 до 41 года. Кроме того, набор данных включает аннотации для детей и пожилых людей.

Учитывая наличие вложенных фотографий, оценить возраст моделей будет непросто. На изображениях присутствуют предметы, закрывающие лицо, например руки, жевательная резинка, очки, головные уборы и т. д.

## Обучение модели

Перенесите сюда код обучения модели и её результат вывода на экран.


(Код в этом разделе запускается в отдельном GPU-тренажёре, поэтому оформлен не как ячейка с кодом, а как код в текстовой ячейке)

```python

# <Код модели
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import tensorflow as tf


def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    datagen = ImageDataGenerator(validation_split=0.25, horizontal_flip=True, rescale=1./255)
    train_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)

    return train_gen_flow

def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')
    datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)
    test_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)

    return test_gen_flow


def create_model(input_shape):

    backbone = ResNet50(input_shape=input_shape,
                    weights='imagenet', 
                    include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))

    optimizer = Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error',
                  metrics=['mae'])


    return model


def train_model(model, train_data, test_data, batch_size=None, epochs=10,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, 
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model >

```

```

# < Found 5694 validated image filenames.
Found 1897 validated image filenames.

<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 356 steps, validate for 119 steps
Epoch 1/10
2023-08-23 17:27:25.514109: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-08-23 17:27:25.915925: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
356/356 - 52s - loss: 207.9160 - mae: 10.4607 - val_loss: 434.4070 - val_mae: 15.6134
Epoch 2/10
356/356 - 43s - loss: 87.1638 - mae: 7.0480 - val_loss: 113.0580 - val_mae: 8.2503
Epoch 3/10
356/356 - 43s - loss: 58.7303 - mae: 5.8235 - val_loss: 92.6808 - val_mae: 7.4397
Epoch 4/10
356/356 - 43s - loss: 40.9481 - mae: 4.9370 - val_loss: 67.9666 - val_mae: 6.0543
Epoch 5/10
356/356 - 44s - loss: 32.1568 - mae: 4.3472 - val_loss: 71.6548 - val_mae: 6.4892
Epoch 6/10
356/356 - 71s - loss: 27.1837 - mae: 4.0074 - val_loss: 107.4730 - val_mae: 8.3191
Epoch 7/10
356/356 - 65s - loss: 20.0870 - mae: 3.4116 - val_loss: 67.1939 - val_mae: 6.1766
Epoch 8/10
356/356 - 69s - loss: 16.7014 - mae: 3.1496 - val_loss: 89.3059 - val_mae: 7.3904
Epoch 9/10
356/356 - 70s - loss: 14.6178 - mae: 2.9173 - val_loss: 69.7169 - val_mae: 6.3712
Epoch 10/10
356/356 - 70s - loss: 12.5354 - mae: 2.7217 - val_loss: 62.7269 - val_mae: 5.8523
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
119/119 - 12s - loss: 62.7269 - mae: 5.8523
Test MAE: 5.8523 >

```

Модель сверточной нейронной сети была построена с использованием архитектуры ResNet50.

После прохождения 10 периодов обучения в наборе обучающих данных было достигнуто заметное снижение средней абсолютной ошибки (MAE) с 10 до 2,8. Продолжение обучения обещает дальнейшее улучшение этих результатов.

## Анализ обученной модели

***Вывод***    
Обученная модель демонстрирует впечатляющую способность точно предсказывать возраст клиентов. Достижение значения средней абсолютной ошибки (MAE), равного 5,85 считается весьма похвальным, особенно из-за того, что данное расхождение приемлемо в пределах возрастных категорий пользователей. Такое признание обусловлено, главным образом, практикой группировки возрастных категорий клиентов по интервалам 8–12 лет.

Использование графических процессоров и нейронных сетей способствует быстрой и эффективной обработке больших объемов данных, сохраняя при этом высокую степень точности.